<a href="https://colab.research.google.com/github/XenoicZ/EPE/blob/main/GNN_ref.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/atlas-calo-ml/gn4pions_eastbay.git

Cloning into 'gn4pions_eastbay'...
remote: Enumerating objects: 1068, done.
remote: Counting objects: 100% (285/285), done.
remote: Compressing objects: 100% (189/189), done.
remote: Total 1068 (delta 163), reused 198 (delta 95), pack-reused 783
Receiving objects: 100% (1068/1068), 195.30 MiB | 41.70 MiB/s, done.
Resolving deltas: 100% (615/615), done.


In [ ]:

!cd gn4pions_eastbay
!pip install uproot
!pip install graph_nets
!pip install sonnet>=2.0.0
#!pip install dm-sonnet
#!pip install graph_nets "tensorflow_gpu>=1.15,<2" "dm-sonnet<2" "tensorflow_probability<0.9"
!pip install graphs
!pip install compress_pickle
!pip install PyYAML==5.1

TensorFlow 1.x selected.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%cd gn4pions_eastbay

/content/gn4pions_eastbay


In [ ]:
import importlib
import numpy as np
import os
import sys
import glob
import uproot as ur
import matplotlib.pyplot as plt
import time
import seaborn as sns
import tensorflow as tf


#from graph_nets.graphs import GraphsTuple
#sys.path.insert(1, '/content/drive/MyDrive/ml4pion')
#utils_np = importlib.import_module("/content/drive/MyDrive/ml4pion/graph_nets/utils_np.py")
#utils_tf = importlib.import_module("/content/drive/MyDrive/ml4pion/graph_nets/utils_tf.py")
#GraphsTuple = importlib.import_module("/content/drive/MyDrive/ml4pion/graph_nets/graphs.py")

import sonnet as snt
import argparse
#!pip install --ignore-installed PyYAML
import logging
import tensorflow as tf

from gn4pions.modules.data import GraphDataGenerator
from gn4pions.modules.models import MultiOutWeightedRegressModel
from gn4pions.modules.utils import convert_to_tuple

sns.set_context('poster')

import graph_nets
from graph_nets import utils_np
from graph_nets import utils_tf
import yaml

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-3d97a0f34d0d>", line 19, in <module>
    import sonnet as snt
  File "/usr/local/lib/python3.7/dist-packages/sonnet/__init__.py", line 21, in <module>
    from sonnet import distribute
  File "/usr/local/lib/python3.7/dist-packages/sonnet/distribute.py", line 21, in <module>
    from sonnet.src.distribute.batch_norm import CrossReplicaBatchNorm
  File "/usr/local/lib/python3.7/dist-packages/sonnet/src/distribute/batch_norm.py", line 22, in <module>
    from sonnet.src import batch_norm
  File "/usr/local/lib/python3.7/dist-packages/sonnet/src/batch_norm.py", line 34, in <module>
    class BaseBatchNorm(base.Module):
  File "/usr/local/lib/python3.7/dist-packages/sonnet/src/batch_norm.py", line 137, in BaseBatchNorm
    offset: Optional[tf.Tensor] = None):
  File "/usr/lo

ImportError: ignored

In [ ]:
# Loading model config
config_file = '/content/gn4pions_eastbay/gn4pions/configs/weightedRegress.yaml'
config = yaml.load(open(config_file), Loader=yaml.FullLoader)


# Data config
data_config = config['data']

data_dir = data_config['data_dir']
num_train_files = data_config['num_train_files']
num_val_files = data_config['num_val_files']
batch_size = data_config['batch_size']
shuffle = data_config['shuffle']
num_procs = data_config['num_procs']
preprocess = data_config['preprocess']
output_dir = '/content/drive/MyDrive/ml4pion/model'
already_preprocessed = data_config['already_preprocessed']  # Set to false when running training for first time


# Model Config
model_config = config['model']

concat_input = model_config['concat_input']


# Traning Config
train_config = config['training']

epochs = train_config['epochs']
learning_rate = train_config['learning_rate']
alpha = train_config['alpha']
os.environ['CUDA_VISIBLE_DEVICES'] = str(train_config['gpu'])
log_freq = train_config['log_freq']
save_dir = train_config['save_dir'] + config_file.replace('.yaml','').split('/')[-1] + '_' + time.strftime("%Y%m%d")

os.makedirs(save_dir, exist_ok=True)
yaml.dump(config, open(save_dir + '/config.yaml', 'w'))

In [ ]:
#data_dir = '/content/drive/MyDrive/ml4pion/ML4Pion/PFNet/v01-45/pipm'
#files_names = np.zeros(100)
#for i in range(100):
#  files_names[i] = 
#files_names = data_dir

'/usr/workspace/hip/ML4Jets/regression_images/'

In [ ]:
# Read data and create data generators
data_dir_0 = '/content/drive/MyDrive/ml4pion/data/pion_files/pi0_files/'
data_dir_pm = '/content/drive/MyDrive/ml4pion/data/pion_files/pion_files/'
pi0_files = np.array(0)
pion_files = np.array(0)
for i in range(1,11):
  pi0_files = np.append(pi0_files, data_dir_0 + '0' + str(i+10) + '.npy')
  pion_files = np.append(pion_files, data_dir_pm + '0' + str(i) + '.npy' )

#pi0_files = np.sort(glob.glob(data_dir+'*graphs.v01*/*pi0*/*root'))
#pion_files = np.sort(glob.glob(data_dir+'*graphs.v01*/*pion*/*root'))


num_train_files = 10
num_val_files = 10
train_start = 10
train_end = train_start + num_train_files
val_end = train_end + num_val_files

pi0_train_files = pi0_files[train_start:train_end]
pi0_val_files = pi0_files[train_end:val_end]
pion_train_files = pion_files[train_start:train_end]
pion_val_files = pion_files[train_end:val_end]

train_output_dir = None
val_output_dir = None


already_preprocessed = False

# Get Data
if preprocess:
    train_output_dir = output_dir + '/train/'
    val_output_dir = output_dir + '/val/'

    if already_preprocessed:
        train_files = np.sort(glob.glob(train_output_dir+'*.p'))[:num_train_files]
        val_files = np.sort(glob.glob(val_output_dir+'*.p'))[:num_val_files]

        pi0_train_files = train_files
        pi0_val_files = val_files
        pion_train_files = None
        pion_val_files = None

        train_output_dir = None
        val_output_dir = None

# Traning Data Generator
# Will preprocess data if it doesnt find pickled files
data_gen_train = GraphDataGenerator(pi0_file_list=pi0_train_files,
                                    pion_file_list=pion_train_files,
                                    cellGeo_file='/content/drive/MyDrive/ml4pion/data/pion_files/cell_geo.root',
                                    batch_size=batch_size,
                                    shuffle=shuffle,
                                    num_procs=num_procs,
                                    preprocess=preprocess,
                                    output_dir=train_output_dir)

# Validation Data generator
# Will preprocess data if it doesnt find pickled files
data_gen_val = GraphDataGenerator(pi0_file_list=pi0_val_files,
                                  pion_file_list=pion_val_files,
                                  cellGeo_file='/content/drive/MyDrive/ml4pion/data/pion_files/cell_geo.root',
                                  batch_size=batch_size,
                                  shuffle=shuffle,
                                  num_procs=num_procs,
                                  preprocess=preprocess,
                                  output_dir=val_output_dir)



Preprocessing and saving data to /content/drive/MyDrive/ml4pion/model/train/
Processing file number 0
Finished processing 0 files

Preprocessing and saving data to /content/drive/MyDrive/ml4pion/model/val/


In [ ]:
# Get batch of data
def get_batch(data_iter):
    for graphs, targets in data_iter:
        graphs = convert_to_tuple(graphs)
        targets = tf.convert_to_tensor(targets)
        yield graphs, targets
        
# Define loss function        
mae_loss = tf.keras.losses.MeanAbsoluteError()
bce_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
def loss_fn(targets, regress_preds, class_preds):
    regress_loss = mae_loss(targets[:,:1], regress_preds)
    class_loss = bce_loss(targets[:,1:], class_preds)
    combined_loss = alpha*regress_loss + (1 - alpha)*class_loss 
    return regress_loss, class_loss, combined_loss

In [ ]:
data_gen_train

In [ ]:
getattr(samp_graph, 'n_edge')

AttributeError: ignored

In [ ]:
from graph_nets import graphs
graphs.ALL_FIELDS

('nodes', 'edges', 'receivers', 'senders', 'globals', 'n_node', 'n_edge')

In [ ]:
# Get a sample graph for tf.function decorator
#samp_graph, samp_target = next(get_batch(data_gen_train.generator()))
#data_gen_train.kill_procs()
#graph_spec = utils_tf.specs_from_graphs_tuple(samp_graph, True, True, True)


# Traning set
#@tf.function(input_signature=[graph_spec, tf.TensorSpec(shape=[None,2], dtype=tf.float32)])
def train_step(graphs, targets):
    with tf.GradientTape() as tape:
        regress_output, class_output = model(graphs)
        regress_preds = regress_output.globals
        class_preds = class_output.globals
        regress_loss, class_loss, loss = loss_fn(targets, regress_preds, class_preds)

    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return regress_loss, class_loss, loss


# Validation Step
#@tf.function(input_signature=[graph_spec, tf.TensorSpec(shape=[None,2], dtype=tf.float32)])
def val_step(graphs, targets):
    regress_output, class_output = model(graphs)
    regress_preds = regress_output.globals
    class_preds = class_output.globals
    regress_loss, class_loss, loss = loss_fn(targets, regress_preds, class_preds)
    return regress_loss, class_loss, loss, regress_preds, class_preds

In [ ]:
# Model 
model = MultiOutWeightedRegressModel(global_output_size=1, num_outputs=2, model_config=model_config)

# Optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate)

# Average epoch losses
training_loss_epoch = []
training_loss_regress_epoch = []
training_loss_class_epoch = []
val_loss_epoch = []
val_loss_regress_epoch = []
val_loss_class_epoch = []

# Model checkpointing, load latest model if available
checkpoint = tf.train.Checkpoint(module=model)
checkpoint_prefix = os.path.join(save_dir, 'latest_model')
latest = tf.train.latest_checkpoint(save_dir)
if latest is not None:
    checkpoint.restore(latest)
else:
    checkpoint.save(checkpoint_prefix)

TypeError: ignored

In [ ]:
class MyMLP(snt.Module):
   def __init__(self, name=None):
     super(MyMLP, self).__init__(name=name)
     self.hidden1 = snt.Linear(1024, name="hidden1")
     self.output = snt.Linear(10, name="output")

   def __call__(self, x):
     x = self.hidden1(x)
     x = tf.nn.relu(x)
     x = self.output(x)
     return x

In [ ]:
import inspect
lines = inspect.getsource(sonnet.Module)
print(lines)

AttributeError: ignored